In [ ]:
#@markdown ## 구글드라이브 연동
#@markdown **생성한 이미지를 구글드라이브에 자동저장하려면 체크해주세요**
Google_Drive = False #@param {type:"boolean"}

#@markdown **초기모델(checkpoint)을 구글드라이브에 저장하려면 체크해주세요**
#@markdown <div><font color="red">모델 파일사이즈(2GB ~ 7GB)가 크니 구글드라이브 용량을 확인해주세요</div>
#@markdown <div><font color="red">대신 한번받으면 다음 실행부터는 빨라져요</div>
Checkpoint_Google_Save = False #@param {type:"boolean"}

#@markdown **WebUI Version**
TagVersion = 'v1.6.0' #@param ["v1.3.2", "v1.4.1", "v1.5.2", "v1.6.0"]

#@markdown **폴더 설명** 
#@markdown * install/outputs : 생성된 이미지들이 저장되요
#@markdown * install/models/Stable-diffusion/ : 모델(checkpoint)를 넣어주면 읽어올수 있어요
#@markdown * install/models/Lora : 로라(LoRA)를 넣어주면 읽어올수 있어요
#@markdown * install/models/LyCORIS : 라이코리스(LyCORIS)를 넣어주면 읽어올수 있어요
#@markdown * install/embeddings : 임베딩(Textual Inversion)를 넣어주면 읽어올수 있어요
#@markdown * install/hypernetworks : 하이퍼워크(Hyperworks)를 넣어주면 읽어올수 있어요

#@markdown **초기모델(checkpoint) URL**
Checkpoint_Url = '#template_checkpoint_default#' #@param {type:"string"}
#@markdown **초기모델 파일명**
Checkpoint_Filename = '#template_checkpoint_default_name#' #@param {type:"string"}

#@markdown -----
#@markdown *추가익스텐션*

#@markdown **ControlNet을 사용하지 않으면 체크를 해제해주세요**
ControlNet = True #@param {type:"boolean"}

#@markdown **Dectect Detailer(ddetailer)를 사용하지 않으면 체크를 해제해주세요**
DDetailer = True #@param {type:"boolean"}

#@markdown **After Detailer(adetailer)를 사용하지 않으면 체크를 해제해주세요**
ADetailer = False #@param {type:"boolean"}

#@markdown **Segment Anything을 사용하지 않으면 체크를 해제해주세요**
SegmentAnything = False #@param {type:"boolean"}

NotebookVersion = 'direct'
Workspace = 'ui'

if Google_Drive:
  from google.colab import drive
  drive.mount('/content/drive')

Checkpoint_SavePath = f'/content/{Workspace}/models/Stable-diffusion'
if Google_Drive and Checkpoint_Google_Save:
  Checkpoint_SavePath = f'/content/{Workspace}/models/Stable-diffusion/google'

!apt -y install -qq aria2
!pip install -q xformers==0.0.20 triton==2.0.0 -U

%cd /content
!wget https://raw.githubusercontent.com/neuralninja22/colab/master/misc/install_nightly.sh -O /content/install.sh
!bash /content/install.sh {Workspace} {TagVersion} {ControlNet} {DDetailer} {ADetailer} {SegmentAnything} False

%cd /content/{Workspace}

# checkpoint select
!sed -i -e 's/"sd_model_checkpoint": "",/"sd_model_checkpoint": "{Checkpoint_Filename}",/g' /content/{Workspace}/config.json

# upscaler
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/{Workspace}/models/ESRGAN -o 4x-UltraSharp.pth

# checkpoint
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{Checkpoint_Url}" -d {Checkpoint_SavePath} -o {Checkpoint_Filename}

# vae
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {Checkpoint_SavePath} -o vae-ft-mse-840000-ema-pruned.vae.pt

# lora
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/62833?type=Model&format=SafeTensor" -d /content/{Workspace}/models/Lora -o add_detail.safetensors

# embeddings
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.pt -d /content/{Workspace}/embeddings -o EasyNegative.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/AsciiP/badhandv4/resolve/main/badhandv4.pt -d /content/{Workspace}/embeddings -o badhandv4.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/60938?type=Negative&format=Other" -d /content/{Workspace}/embeddings/ -o negative_hand.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/60095?type=Negative&format=Other" -d /content/{Workspace}/embeddings/ -o bad_prompt_version2.pt

!wget https://raw.githubusercontent.com/neuralninja22/colab/master/misc/direct/v1.6.0/directui.py -O /content/drive/MyDrive/install/directui.py

import sys

sys.argv = [
    sys.argv[0],
    "--xformers",
    "--no-half-vae",
    "--theme",
    "dark"
]

from modules import launch_utils
launch_utils.prepare_environment()
print("아래 코드를 실행해주세요!")


In [ ]:
# 직접실행 (검은색 화면만 보이면 스크롤을 올려보세요)

from directui import webui
webui()